In [ ]:
import socket

# Create a UDP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))

print("UDP Chat Server is ready to receive messages...")

# Dictionary to store client addresses and authentication status
clients = {}

# Hardcoded user credentials (for demonstration purposes)
users = {
    "user1": "password1",
    "user2": "password2"
}

def authenticate(username, password):
    """Check if the username and password are valid."""
    return username in users and users[username] == password

while True:
    # Receive data from a client
    data, client_address = server_socket.recvfrom(2048)
    message = data.decode()

    if client_address not in clients:
        # New client: ask for authentication
        print(f"New client connected: {client_address}")
        server_socket.sendto(b"USERNAME", client_address)  # Ask for username
        username, _ = server_socket.recvfrom(2048)
        server_socket.sendto(b"PASSWORD", client_address)  # Ask for password
        password, _ = server_socket.recvfrom(2048)

        # Authenticate the client
        if authenticate(username.decode(), password.decode()):
            clients[client_address] = {"username": username.decode(), "authenticated": True}
            server_socket.sendto(b"Authentication successful! You can now chat.", client_address)
            print(f"Client {client_address} authenticated as {username.decode()}")
        else:
            server_socket.sendto(b"Authentication failed. Disconnecting...", client_address)
            print(f"Client {client_address} failed authentication.")
            continue
    else:
        # Existing client: check if authenticated
        if not clients[client_address]["authenticated"]:
            server_socket.sendto(b"Please authenticate first.", client_address)
            continue

    # Handle chat messages
    print(f"{clients[client_address]['username']}: {message}")

    # Send a reply to the client
    reply = input("You: ")
    server_socket.sendto(reply.encode(), client_address)

UDP Chat Server is ready to receive messages...
